## Fetch Data From Property Assessment  

### Get Database id by year

In [1]:
import data.sqlDataFetch as sdf

/Users/jingbowang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [232]:
database_id = sdf.get_id_by_year(2005)

### Describe The Database

Because of each database have diff key value, I can not use for loop to get key value

In [233]:
sdf.describe_database(database_id)

Column Name          | Type      
--------------------------------
_id                  | int       
PID                  | text      
CM_ID                | text      
ST_NUM               | text      
ST_NAME              | text      
ST_NAME_SFX          | text      
UNIT_NUM             | text      
ZIPCODE              | text      
PTYPE                | text      
LU                   | text      
OWN_OCC              | text      
OWNER FY04           | text      
MAIL_ADDRESS         | text      
MAIL_CITY_STATE      | text      
MAIL_ZIP             | text      
LOTSIZE              | text      
GROSS_AREA           | text      
LIVING _AREA         | text      
FY2004_TOTAL         | text      
FY200_ LAND          | text      
FY2004_BLDG          | text      
GROSS_TAX            | text      
NUM_FLOORS           | text      


In [225]:
import requests
import pandas as pd

### Get Con_unit, Res_unit, Rc_unit for each year

In [234]:
url = "https://data.boston.gov/api/3/action/datastore_search_sql"
# Construct the SQL query to select distinct city names from the table
sql_query = f"""
SELECT
    "ZIPCODE"::text AS ZIPCODE,
    SUM("LIVING_AREA":: FLOAT) as TOTAL_LIVING_AREA
FROM
    "{database_id}"
GROUP BY
    "ZIPCODE"::text
"""
print(sql_query)
params = {"sql": sql_query}

# Send the request
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()

    # Check if there is data in the response
    if data['success'] and 'result' in data and 'records' in data['result']:
        records = data['result']['records']
        # Filter records with ZIP code length greater than 1 and ensure ZIPCODE is not None
        filtered_records = [record for record in records if record['zipcode'] and len(record['zipcode']) > 1]

        # Convert the filtered records into a DataFrame
        df = pd.DataFrame(filtered_records)
    else:
        print("No data found or error in response.")
else:
    print("Failed to fetch data:", response.status_code)



SELECT
    "ZIPCODE"::text AS ZIPCODE,
    SUM("LIVING_AREA":: FLOAT) as TOTAL_LIVING_AREA
FROM
    "5bfe4ca0-71c0-4751-bdcf-dad4d58445e0"
GROUP BY
    "ZIPCODE"::text

Failed to fetch data: 409


In [235]:
neighborhood_zip_map = {
    'Allston/Brighton': ['02134', '02135', '02163'],
    'Back Bay/Beacon Hill': ['02108', '02116', '02117', '02123', '02133', '02199', '02216', '02217', '02295'],
    'Central Boston': [
        '02101', '02102', '02103', '02104', '02105', '02106', '02107', '02109', '02110', '02111',
        '02112', '02113', '02114', '02196', '02201', '02202', '02203', '02204', '02205', '02206',
        '02207', '02208', '02209', '02211', '02212', '02222', '02293'
    ],
    'Charlestown': ['02129'],
    'Dorchester': ['02122', '02124', '02125'],
    'East Boston': ['02128', '02228'],
    'Fenway/Kenmore': ['02115', '02215'],
    'Hyde Park': ['02136'],
    'Jamaica Plain': ['02130'],
    'Mattapan': ['02126'],
    'Roslindale': ['02131'],
    'Roxbury': ['02119', '02120', '02121'],
    'South Boston': ['02127', '02210', '02219'],
    'South End': ['02118'],
    'West Roxbury': ['02132'],
    'Dedham':['02026', '02137'],
    ' Brookline':['02445', '02446', '02467', '02146'],
    'Newton':['02458'],
    ' Hingham':['02018'],
    'Milton':['02186'],
    'Westwood':['02090']
}


In [236]:
df['zipcode'] = df['zipcode'].astype(str).str.zfill(5)

In [237]:
df['zipcode'] = df['zipcode'].str.rstrip('_')

In [238]:

zip_neighborhood_map = {zip_code: neighborhood for neighborhood, zip_codes in neighborhood_zip_map.items() for zip_code in zip_codes}

df['neighborhood'] = df['zipcode'].map(zip_neighborhood_map)


In [239]:
#df['zipcode'] = df['zipcode'].str.replace('_', '', regex=False)
print(df)

   zipcode  total_living_area  neighborhood
0    01116             1291.0           NaN
1    01224             1470.0           NaN
2    01473              601.0           NaN
3    01720             1353.0           NaN
4    01749              540.0           NaN
..     ...                ...           ...
76   02127             1325.0  South Boston
77   02131              591.0    Roslindale
78   03070                0.0           NaN
79   98221              789.0           NaN
80   0009R                0.0           NaN

[81 rows x 3 columns]


### save each cvs files

In [204]:
import os

save_path = './data/cleaned/'
file_name = '2009.csv'
full_path = os.path.join(save_path, file_name)

if not os.path.exists(save_path):
  os.makedirs(save_path)

df.to_csv(full_path, index=False)

In [ ]:
import pandas as pd
import os

# 定义保存路径
save_path = './data/cleaned/'

# 定义需要读取的年份范围
years = list(range(2009, 2024))  # 生成从2009到2023的列表
years.remove(2014)  # 从列表中移除2014
years.append(2024)  # 添加2024

# 初始化一个空的DataFrame用于存储合并后的数据
combined_df = pd.DataFrame()

# 遍历每一个年份，读取相应的CSV文件
for year in years:
    file_name = f'{year}.csv'
    full_path = os.path.join(save_path, file_name)

    # 读取CSV文件
    temp_df = pd.read_csv(full_path)

    # 添加一个新列用于标记年份
    temp_df['Year'] = year

    # 将当前年份的数据添加到合并后的DataFrame中
    combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

# 保存合并后带有年份标记的数据到新的CSV文件
output_file_name = 'combined_2009_2024.csv'
output_full_path = os.path.join(save_path, output_file_name)
combined_df.to_csv(output_full_path, index=False)
